# Agent fun slippage Adjustment

Problem statement.

Agent.fun implements an nuleash function.

The unleash function requires that slipage be <3%

Due to the sucess of the AGNT ticker, which has accrued 139ETH

The price impact of the purchase is expected to be signficantly greater than this.



balancer_pool = "https://balancer.fi/pools/base/v2/0x5332584890d6e415a6dc910254d6430b8aab7e69000200000000000000000103"

In [1]:
olas_address = '0x54330d28ca3357f294334bdc454a032e7f353416'
usdc_address = '0x833589fcd6edb6e08f4c7c32d4f71b54bda02913'

In [2]:
token_a_pool_amt = 177350
token_b_pool_amt = 290120

cur_price = token_b_pool_amt / token_a_pool_amt
cur_price

1.6358612912320272

In [3]:
eth_price = 3400
eth_qty = 139
eth_in_usd = eth_price * eth_qty
eth_in_usd

472600

In [7]:
token_a_pool_amt = 177350
token_b_pool_amt = 290120

cur_price = token_b_pool_amt / token_a_pool_amt
cur_price

# pulled from the final step such that we can verify the amount
additional_funds = 17893116

def calculate_price_impact_with_fee(reserve_token_a, reserve_token_b, token_b_amount, fee_rate=0.003):
    # Effective token_b amount after fee
    effective_token_b = token_b_amount * (1 - fee_rate)
    
    # Current price of token_a in terms of token_b
    p_current = reserve_token_b / reserve_token_a
    
    # Token_a received during the trade
    token_a_received = reserve_token_a - (reserve_token_a * reserve_token_b) / (reserve_token_b + effective_token_b)
    
    # Average execution price
    p_avg = token_b_amount / token_a_received  # Using the full token_b_amount (pre-fee) for effective price
    
    # Price impact
    price_impact = ((p_avg - p_current) / p_current) * 100

    return price_impact

# Example usage:
reserve_a = token_a_pool_amt  # Example reserve of token_a
reserve_b = token_b_pool_amt + additional_funds # Example reserve of token_b
trade_amount_b = eth_in_usd  # Amount of token_b to spend
fee_rate = 0.003  # 0.3% fee

impact = calculate_price_impact_with_fee(reserve_a, reserve_b, trade_amount_b, fee_rate)
print(f"Price Impact: {impact:.2f}%")


Price Impact: 2.90%


In [8]:
def calculate_required_liquidity(reserve_token_a, reserve_token_b, token_b_amount, fee_rate=0.003, impact_limit=3.0):

    # Binary search to find the additional liquidity needed
    additional_liquidity_low, additional_liquidity_high = 0, 1e12  # Arbitrarily large upper bound
    required_liquidity = 0

    while additional_liquidity_high - additional_liquidity_low > 1e-6:  # Precision threshold
        mid = (additional_liquidity_low + additional_liquidity_high) / 2
        new_reserve_b = reserve_token_b + mid
        impact = calculate_price_impact_with_fee(reserve_token_a, new_reserve_b, token_b_amount, fee_rate)
        if impact < impact_limit:
            required_liquidity = mid
            additional_liquidity_high = mid
        else:
            additional_liquidity_low = mid

    return required_liquidity

# Example usage:
reserve_a = 177350  # Reserve of token_a (olas)
reserve_b = 290120  # Reserve of token_b (e.g., USD)
trade_amount_b = eth_in_usd  # Amount of token_b to spend
fee_rate = 0.003  # 0.3% fee
impact_limit = 2.9  # Desired price impact limit (<3%)

required_liquidity = calculate_required_liquidity(reserve_a, reserve_b, trade_amount_b, fee_rate, impact_limit)
print(f"Required Additional Liquidity of Token B: {required_liquidity:.6f}")


Required Additional Liquidity of Token B: 17893116.213486


In [55]:
# Flashloans

In [58]:
# in order to get this liqudity we can look to borrow from AAVE and Balancer

## AAVE Pools

Aave has two markets we can look at;

https://app.aave.com/reserve-overview/?underlyingAsset=0x833589fcd6edb6e08f4c7c32d4f71b54bda02913&marketName=proto_base_v3

This has 8.13m Available directly as USDC



## Balancer pools
balancer has an interesting factor whereby all assets in all pools are accessible as unified source of liqudity.

Theoretically this would allow us to access all the usdc on the platform for the purpose of the flash

This would enable us to access;

At a minimum (we are only looking at top 3 pools for the purpose of this);

3.76m https://balancer.fi/pools/base/v2/0x59501a303b1bdf5217617745acec4d99107383f0000200000000000000000197
290k https://balancer.fi/pools/base/v2/0x5332584890d6e415a6dc910254d6430b8aab7e69000200000000000000000103
165k https://balancer.fi/pools/base/v2/0x6c994a2ce94c36242669780804e9d5f2bdd96a980000000000000000000001a0

4.2m 

# Morpho Blue
https://docs.morpho.org/morpho/tutorials/free-flash-loans

morpho blue also offer free flash loans

They have roughly 4m USDC of liquidity available

In [68]:
aave_usdc = 8130000
bal_usdc   = 4200000
morpho_usdc = 4_000_000
shortfall = additional_funds - aave_usdc - bal_usdc - morpho_usdc
print(f"Shortfall USDC: {shortfall}")

print(f"Shortfall in ETH: {shortfall / eth_price}")

Shortfall USDC: 1563116
Shortfall in ETH: 459.74


## Shortfall

We are unable to locate further USDC flashloanable assets, however we can use aerodrome to swap borrowed ETH to usd and back post unleashing

https://app.aave.com/reserve-overview/?underlyingAsset=0x4200000000000000000000000000000000000006&marketName=proto_base_v3

There is 17m of liquidity as ETH in this pool

We lso have the balncer WETh avilable.



In [72]:

## Proposed FlashContract

class AgentFunContract:
    usdc = 0
    weth = 0
    olas = 0

    def save_our_bacon(self):
        self.usdc += self.bal_usdc_flash() 
        self.usdc += self.aave_usdc_flash()
        self.usdc += self.morpho_usdc_flash()

        self.weth += self.bal_weth_flash(shortfall)
        self.swap_weth_usdc()

        # we now have all the funds we need, in order to sandwich the unleash event, we will purchase some OLAS
        self.olas += bal.swap("usdc", "olas") # not quite sure what this should be optimally, or even to ensure our profit covers costs of all of this


        bal.join(OLAS_POOL, self.usdc) # we ensure that we just provide the USDC here
        self.usdc = 0
        agent_fun.unleash()
        # we know the price will have now pumped
        # we need to withdraw our tokens;
        usdc_amt, olas_amt = bal.leave(OLAS_POOL) # ensure we take out all lp
        # we have to consider that we will get some olas which we need to sell to repay all lps. we are likely to get about 95% USDC with the rest as olas.
        self.olas += olas_amt
        # We sell all our olas
        bal.sell_olas_to_usdc(self.olas)

        # we repay all our loans
        self.repay()

        
    def repay(self):
        # we swap from USDC to weth
        aerodrome.getExactTokensOut("USDC", "WETH", self.weth_liability)
        bal.repay_weth(self.weth_liability)
        # We repay the usdc
        bal.repay_usdc(self.bal_usdc_liability)

        # we repay aave
        aave.repay_usdc(self.aave_liability)

        # we repay morpho
        morpho.repay(self.morph_liability)
        

    def swap_weth_usdc(self):
        self.usdc += aerodrome.swap(self.weth)
        self.weth = 0
        
    def aave_usdc_flash(self):
        self.aave_liability = aave.max_flash(usdc)
        return self.aave_liability
        
    def bal_usdc_flash(self):
        self.bal_usdc_liability =  bal.max_flash(usdc)
        return self.bal_usdc_liability
        
    def morpho_usdc_flash(self):
        self.morpho_liability =  morpho.max_flash(usdc)
        return self.morpho_liability

    def bal_weth_flash(self, required_usdc):
        eth_price = aerodrome.get_price(WETH)
        self.weth_liability = (required_usdc / eth_price) * 1.01 # safety margin of 1%
        return bal.flash("weth", self.weth_liability)


    